In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('./gdrive/MyDrive/AEGAN/AE_GAN_IAENG')
!nvidia-smi
!ls

In [ ]:
!pip install adversarial-robustness-toolbox

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from art.attacks.evasion import FastGradientMethod
from art.attacks.evasion import CarliniL2Method
from art.attacks.evasion import DeepFool
from art.attacks.evasion import BasicIterativeMethod
from art.attacks.evasion import SaliencyMapMethod
from art.estimators.classification import PyTorchClassifier
from art.utils import load_mnist
from art.defences.trainer import AdversarialTrainer
from art.defences.trainer import AdversarialTrainerMadryPGD

In [ ]:
class MnistCNN(nn.Module):

    def __init__(self):
        super(MnistCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.fc3 = nn.Linear(1024, 128)
        self.fc4 = nn.Linear(128, 10)

    def forward(self, x):
        h = F.relu(self.conv1(x))
        h = F.relu(self.conv2(h))
        h = F.dropout2d(F.max_pool2d(h, 6), p=0.25)
        h = F.dropout2d(self.fc3(h.view(h.size(0), -1)), p=0.5)
        h = self.fc4(h)
        return F.log_softmax(h,dim=1)

In [ ]:
(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_mnist()
x_train = np.swapaxes(x_train, 1, 3).astype(np.float32)
x_test = np.swapaxes(x_test, 1, 3).astype(np.float32)

In [ ]:
model = MnistCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
classifier = PyTorchClassifier(
    model=model,
    clip_values=(0, 1),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(1, 28, 28),
    nb_classes=10,
)

In [ ]:
classifier.fit(x_train, y_train, batch_size=128, nb_epochs=30)

In [ ]:
predictions1 = classifier.predict(x_test)
accuracy1 = np.sum(np.argmax(predictions1, axis=1) == np.argmax(y_test, axis=1))/len(y_test)
print('Accuracy1 test examples:{}%'.format(accuracy1*100))

In [ ]:
attack_fgsm1 = FastGradientMethod(estimator=classifier,eps=0.1)
attack_fgsm2 = FastGradientMethod(estimator=classifier,eps=0.3)
attack_fgsm3 = FastGradientMethod(estimator=classifier,eps=0.5)
attack_fgsm4 = FastGradientMethod(estimator=classifier,eps=0.7)
fgsm_at = AdversarialTrainer(classifier, [attack_fgsm1,attack_fgsm2,attack_fgsm3,attack_fgsm4], ratio=0.5)
fgsm_at.fit(x_train, y_train, batch_size=128, nb_epochs=30)

# pgd_at = AdversarialTrainerMadryPGD(classifier, nb_epochs=300, batch_size=128, eps = 8, eps_step = 2, max_iter = 7)
# pgd_at.fit(x_train, y_train)

In [ ]:
predictions2 = classifier.predict(x_test)
accuracy2 = np.sum(np.argmax(predictions2, axis=1) == np.argmax(y_test, axis=1))/len(y_test)
print('Accuracy2 on test examples:{}%'.format(accuracy2*100))

Accuracy2 on test examples:88.01%


In [ ]:
# attack = FastGradientMethod(estimator=classifier,eps=0.1)
# attack = BasicIterativeMethod(estimator=classifier, eps=0.1)
# attack = SaliencyMapMethod(classifier=classifier, theta=0.3, gamma=0.7)
# attack = CarliniL2Method(classifier=classifier, max_iter=500)
# attack = DeepFool(classifier=classifier)                                                
# adv = attack.generate(x=x_test[0:2000])
cw = torch.load('./CW.tar')
adv = cw['CW']
y_test = cw['y']
predictions3 = classifier.predict(adv)
# accuracy3 = np.sum(np.argmax(predictions3, axis=1) == np.argmax(y_test[0:2000], axis=1))/len(y_test[0:2000])
accuracy3 = np.sum(np.argmax(predictions3, axis=1) == y_test.numpy())/len(y_test)
print('Accuracy3 on test examples:{}%'.format(accuracy3*100))

Accuracy3 on test examples:10.9375%
